<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Dataforst%C3%A5else-og-unders%C3%B8kelse-Pradeep/Bioco_Databehandling_av_manglende_verdier_og_utstikkere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige pakker og bibliotek

In [1]:
! pip install pyod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.1.2-py3-none-any.whl size=190292 sha256=3b25192cc965da1d46d65e2437cda01f40b5839cc9db8ed97a7beea499800609
  Stored in directory: /root/.cache/pip/wheels/81/1b/61/aa85b78c3c0c8871f4231e3f4a03bb23cecb7db829498380ee
Successfully built pyod


Importering av nødvendige pakker og bibliotek

In [20]:
import pandas as pd
import numpy as np
import copy
from datetime import datetime

from pyod.models.hbos import HBOS

Importering av data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Velger første kolonne med dato og tid som index
bioco_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/førbehandlet_bioco_data.csv',
                         header=0,
                         sep=',',
                         index_col=0)

# Formatterer index til riktig format og datatype
bioco_data.index = pd.to_datetime(bioco_data.index,
                                  format='%Y-%m-%d %H:%M:%S')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Oppdeling av data til prediktorer- og responsvariabler

In [4]:
# Datasett for prediktorer
prediktor_data = bioco_data.iloc[:, :-4]

# Verdier for prediktorer
X = prediktor_data.values

# Datasett for hver kvalitetsmåling
collagen = bioco_data['Collagen']
mw = bioco_data['Mw']
smallmolecules = bioco_data['SmallMolecules']
brixadjusted = bioco_data['BrixAdjusted']

# Behandling av manglende verdier

Oversikt over antall manglende verider for per kolonne

In [5]:
bioco_data.isnull().sum()

EnzymeType_A1         0
EnzymeType_A2         0
EnzymeType_B          0
EnzymeType_C          0
EnzymeType_D          0
EnzymeType_E          0
RawMatFlow           55
WaterFlow            55
RawMatPercent        55
NIRfat            14187
NIRprotein        15731
NIRash            13976
NIRwater          13958
TT07                 50
TT08                 45
PT03                 25
TT20                 25
TT09                 15
TT12                  0
Collagen          43152
Mw                42711
SmallMolecules    42711
BrixAdjusted      42711
dtype: int64

Oversikt over rader med manglende verdier for RawMatflow

In [6]:
# Henter ut index til rader med manglende verdier for "RawMatFlow"
rawmat_manglende_index = bioco_data[bioco_data['RawMatFlow'].isnull()].index

# Sjekker hvilke andre kolonner som har manglende verdier for samme rader
bioco_data[rawmat_manglende_index[0]:rawmat_manglende_index[-1]].isnull().sum()

# Dropper de spesifikke rader fra datasett
bioco_data.drop(bioco_data.loc[rawmat_manglende_index[0]:rawmat_manglende_index[-1]].index,
                inplace=True)

Oppdeling av datasett for med og uten NIR

In [49]:
# Lager en kopi av data for uten NIR målinger
data_utenNIR = copy.deepcopy(bioco_data)

# Utelukker kolonner med NIR målinger fra nye datasettet
data_utenNIR.drop(columns=['NIRfat',
                           'NIRprotein',
                           'NIRash',
                           'NIRwater'],
                  inplace=True)

Oversikt over rader med manglende verdier for NIRfat

In [82]:
# Henter ut index til rader med manglende verdier for "NIRfat"
nirfat_manglende_index = bioco_data[bioco_data['NIRfat'].isnull()].index

# Lager liste for å lagre start og slutttid for serie med manglende verdier
nirfat_manglende_index_start = list()
nirfat_manglende_index_slutt = list()

# Legger til starttid for første serie med manglende verdier
nirfat_manglende_index_start.append(nirfat_manglende_index[0])

# Lagrer alle start og slutttid for serier med manglende verdier
for i in range(1, len(nirfat_manglende_index)):
  tidsdiff = nirfat_manglende_index[i] - nirfat_manglende_index[i-1]

  if tidsdiff.total_seconds() > 60:
    nirfat_manglende_index_slutt.append(nirfat_manglende_index[i-1])
    nirfat_manglende_index_start.append(nirfat_manglende_index[i])
nirfat_manglende_index_slutt.append(nirfat_manglende_index[-1])

for i,j in zip(nirfat_manglende_index_start, nirfat_manglende_index_slutt):
  tidsdiff = ((j-i).total_seconds())/60 + 1
  print("Tidsdiff", tidsdiff)
  nirfat_mangel = bioco_data[i:j].isnull().sum()['NIRfat']
  respons_mangel = bioco_data[i:j].isnull().sum()[['Collagen', 'Mw', 'SmallMolecules', 'BrixAdjusted']]
  if nirfat_mangel > min(respons_mangel.values):
    print("Tidsintervallet {} til {} har flere manglende NIRfat målinger enn kvalitetsmålinger".format(i, j))
    print("NIRfat {}".format(nirfat_mangel))
    print(respons_mangel)
  else:
    print("Tidsintervallet {} til {} har like manglende NIRfat målinger som kvalitetsmålinger".format(i, j))

Tidsdiff 127.0
Tidsintervallet 2022-10-31 15:30:00 til 2022-10-31 17:36:00 har like manglende NIRfat målinger som kvalitetsmålinger
Tidsdiff 13.0
Tidsintervallet 2022-11-02 05:07:00 til 2022-11-02 05:19:00 har like manglende NIRfat målinger som kvalitetsmålinger
Tidsdiff 10.0
Tidsintervallet 2022-11-02 10:47:00 til 2022-11-02 10:56:00 har like manglende NIRfat målinger som kvalitetsmålinger
Tidsdiff 35.0
Tidsintervallet 2022-11-03 01:59:00 til 2022-11-03 02:33:00 har like manglende NIRfat målinger som kvalitetsmålinger
Tidsdiff 537.0
Tidsintervallet 2022-11-04 15:08:00 til 2022-11-05 00:04:00 har flere manglende NIRfat målinger enn kvalitetsmålinger
NIRfat 537
Collagen          537
Mw                536
SmallMolecules    536
BrixAdjusted      536
dtype: int64
Tidsdiff 16.0
Tidsintervallet 2022-11-07 16:24:00 til 2022-11-07 16:39:00 har like manglende NIRfat målinger som kvalitetsmålinger
Tidsdiff 65.0
Tidsintervallet 2022-11-08 22:15:00 til 2022-11-08 23:19:00 har like manglende NIRfat

Dropper rader med manglende verdier for NIRfat, kun der kvalitetsmålinger også har manglende verdier

In [73]:
data_prosessert = bioco_data[~(bioco_data['NIRfat'].isna() & bioco_data[['Collagen', 'Mw', 'SmallMolecules', 'BrixAdjusted']].isna().all(axis=1))]

Behandling av resterende manglende verdier

# Inspeksjon av mulige ekstremverdier

## Ekstremverdi inspeksjon med HBOS

In [ ]:
hbos = HBOS()
hbos.fit(X)

y_predikert = hbos.labels_
y_scores = hbos.decision_scores_

ValueError: Input contains NaN.

Oppdeling av original datasett for hver kvalitetsmåling

In [ ]:
# Trekker ut prediktorer som et eget datasett
prediktor_data = bioco_rå_data.iloc[:, :-4]

# Datasett for "Collagen"
collagen_datasett = pd.concat([prediktor,
                               bioco_rå_data['Collagen']],
                              axis=1)

# Datasett for "Mw"
mw_datasett = pd.concat([prediktor,
                         bioco_rå_data['Mw']],
                        axis=1)

# Datasett for "SmallMolecules"
smallmolecules_datasett = pd.concat([prediktor,
                                     bioco_rå_data['SmallMolecules']],
                                    axis=1)

# Datasett for "BrixAdjusted"
brixadjusted_datasett = pd.concat([prediktor,
                                   bioco_rå_data['BrixAdjusted']],
                                  axis=1)

Lagrer hvert datasett som csv

In [ ]:
# Collagen datasett som csv fil
collagen_datasett.to_csv('')